# 03. Training Demonstration (Sim-to-Real)

### The Fine-Tuning Process
To bridge the gap between simulation and reality, we use **Transfer Learning**.
1.  **Weights:** We start with the `yolov8s-worldv2` weights (pre-trained on massive datasets like Objects365).
2.  **Data:** We feed it our `Simulated` images (Source Domain).
3.  **Validation:** We monitor performance on `Real` images (Target Domain).

### Note
This notebook runs a **minimal demo** (1 Epoch, Batch size 2) just to verify that the training pipeline works within Jupyter. For the actual training, please use the `src/train_domain_adapt.py` script in the terminal for better performance and stability.

In [ ]:
from ultralytics import YOLOWorld
import config
import torch

# Check for GPU
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Training on: {torch.cuda.get_device_name(0) if device == 0 else 'CPU'}")

# Load Model
model = YOLOWorld(config.MODEL_NAME)

In [ ]:
# Start a 'Toy' Training Loop
# This is just to demonstrate functionality
try:
    results = model.train(
        data=str(config.DATA_DIR / 'data.yaml'),
        epochs=1,          # Only 1 epoch for demo
        imgsz=640,
        batch=2,           # Small batch size to avoid OOM
        device=device,
        name='notebook_demo_run',
        project=str(config.REPORTS_DIR / 'runs_demo')
    )
    print("Demo training completed successfully!")
except Exception as e:
    print(f"Training interrupted or failed: {e}")

### Visualizing Loss
If the training runs successfully, we can plot the loss curves to see if the model is converging (learning).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Path to the results CSV from the demo run
csv_path = config.REPORTS_DIR / 'runs_demo/notebook_demo_run/results.csv'

if csv_path.exists():
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()
    
    plt.figure(figsize=(10, 5))
    plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss', marker='o')
    plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss', marker='o')
    plt.title("Demo Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
else:
    print("No results file found (Training might have failed or not started).")